In [ ]:
!pip install polars

In [14]:
import torch
torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)
# torch.cuda.is_available()
# torch.cuda.device_count()
# torch.cuda.current_device()
# torch.cuda.get_device_name(0)

In [24]:
import torch
import gc

# Delete all references to models
for obj in list(globals().values()):
    if torch.is_tensor(obj) or isinstance(obj, torch.nn.Module):
        del obj

# Collect garbage
gc.collect()

# Free up all GPU memory
for i in range(torch.cuda.device_count()):
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats(i)

print("Memory cleared on all GPUs.")

Memory cleared on all GPUs.


In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [26]:
# !kill -9 1633693 1633694 1633695

In [27]:
!nvidia-smi

Tue Oct 29 16:39:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off |   00000000:67:00.0 Off |                  N/A |
| 27%   28C    P8             24W /  250W |       2MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [28]:

import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import math
from glob import glob
from pathlib import Path
import joblib
import pickle
import itertools
from tqdm.auto import tqdm
import re
import torch
import vllm
import polars as pl

# Initialize LLM and Tokenizer
llm = vllm.LLM(
    "Qwen/Qwen2.5-Math-7B-Instruct",  # Ensure this model is supported
    tensor_parallel_size=4,  # or 4 based on available resources
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
)
tokenizer = llm.get_tokenizer()

WARNING 10-29 16:40:18 config.py:1668] Casting torch.bfloat16 to torch.float16.
INFO 10-29 16:40:25 config.py:905] Defaulting to use mp for distributed inference
WARNING 10-29 16:40:25 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-29 16:40:25 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='Qwen/Qwen2.5-Math-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-Math-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding

(VllmWorkerProcess pid=1653180) /u/hpm3ep/.local/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
(VllmWorkerProcess pid=1653180)   @torch.library.impl_abstract("xformers_flash::flash_fwd")
(VllmWorkerProcess pid=1653181) /u/hpm3ep/.local/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
(VllmWorkerProcess pid=1653181)   @torch.library.impl_abstract("xformers_flash::flash_fwd")
(VllmWorkerProcess pid=1653182) /u/hpm3ep/.local/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please us

(VllmWorkerProcess pid=1653180) INFO 10-29 16:40:31 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=1653182) INFO 10-29 16:40:31 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=1653181) INFO 10-29 16:40:31 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
INFO 10-29 16:40:31 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=1653180) INFO 10-29 16:40:31 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=1653181) INFO 10-29 16:40:31 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=1653180) INFO 10-29 16:40:31 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=1653181) INFO 10-29 16:40:31 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=1653182) INFO 10-29 16:40:31 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=1653182) INFO 10-29 16:40:31 pynccl.py:63] vLLM is using nccl==2.20.5
I

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-29 16:40:37 model_runner.py:1067] Loading model weights took 3.5478 GB
(VllmWorkerProcess pid=1653181) INFO 10-29 16:40:37 model_runner.py:1067] Loading model weights took 3.5478 GB
(VllmWorkerProcess pid=1653182) INFO 10-29 16:40:37 model_runner.py:1067] Loading model weights took 3.5478 GB
(VllmWorkerProcess pid=1653180) INFO 10-29 16:40:37 model_runner.py:1067] Loading model weights took 3.5478 GB
INFO 10-29 16:40:38 distributed_gpu_executor.py:57] # GPU blocks: 21742, # CPU blocks: 18724
INFO 10-29 16:40:38 distributed_gpu_executor.py:61] Maximum concurrency for 4096 tokens per request: 84.93x


In [29]:
def generate_text_vllm(requests, tokenizer, model):
    sampling_params = vllm.SamplingParams(
        temperature=0.00,
        seed=42, 
        max_tokens=1024
    )
    responses = model.generate(requests, sampling_params=sampling_params, use_tqdm=False)
    response_text_list = []
    for response in responses:
        # total_tokens += len(response.outputs[0].token_ids)
        response_text_list.append(response.outputs[0].text)
    return response_text_list

In [30]:
import os
import tempfile
import subprocess

class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout

    def __call__(self, query):
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            
            try:
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
            except subprocess.TimeoutExpired:
                return False, f"Execution timed out after {self.timeout} seconds."

            if result.returncode == 0:
                output = result.stdout.strip()
                return True, output
            else:
                error_msg = result.stderr.strip()
                # Process the error message to remove the temporary file path
                # This makes the error message cleaner and more user-friendly
                error_lines = error_msg.split("\n")
                cleaned_errors = []
                for line in error_lines:
                    if temp_file_path in line:
                        # Remove the path from the error line
                        line = line.replace(temp_file_path, "<temporary_file>")
                    cleaned_errors.append(line)
                cleaned_error_msg = "\n".join(cleaned_errors)
                return False, cleaned_error_msg

In [31]:
import re


def extract_python_code(text):
    pattern = r'```python\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    return "\n\n".join(matches)


def process_python_code(query):
    query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
    current_rows = query.strip().split("\n")
    new_rows = []
    for row in current_rows:
        new_rows.append(row)
        if not row.startswith(" ") and "=" in row:
                variable_to_print = row.split("=")[0].strip()
                new_rows.append(f'print(f"{{{variable_to_print}=}}")')
    return "\n".join(new_rows)


def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    return matches[0]


from collections import Counter
import random
def select_answer(answers):
    counter = Counter()
    for answer in answers:
        try:
            if int(answer) == float(answer):
                counter[int(answer)] += 1 + random.random() / 1_000
        except:
            pass
    if not counter:
        return 210
    _, answer = sorted([(v,k) for k,v in counter.items()], reverse=True)[0]
    return answer%1000

In [33]:
sampling_params = vllm.SamplingParams(
    temperature=1.0,              # randomness of the sampling
    min_p=0.01,
    skip_special_tokens=True,     # Whether to skip special tokens in the output.
    max_tokens=1800,
    stop=["```\n"],
    include_stop_str_in_output=True,
)

def batch_message_generate(list_of_messages) -> list[list[dict]]:

    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]
    
    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )
    
    for messages, single_request_output in zip(list_of_messages, request_output):
        # print()
        # print(single_request_output.outputs[0].text)
        # print()
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})

    return list_of_messages

In [34]:
def batch_message_filter(list_of_messages) -> tuple[list[list[dict]], list[str]]:
    extracted_answers = []
    list_of_messages_to_keep = []
    for messages in list_of_messages:
        answer = extract_boxed_text(messages[-1]['content'])
        if answer:
            extracted_answers.append(answer)
        else:
            list_of_messages_to_keep.append(messages)
    return list_of_messages_to_keep, extracted_answers

In [35]:
def batch_message_execute(list_of_messages) -> list[list[dict]]:
    for messages in list_of_messages:
        python_code = extract_python_code(messages[-1]['content'])
        python_code = process_python_code(python_code)
        # print('\n\n' + python_code + '\n\n')
        try:
            print('c', end='')
            is_successful, output = PythonREPL()(python_code)
            if is_successful:
                print('o', end='')
            else:
                print('e', end='')
        except Exception as e:
            print('f', end='')
            output = str(e)
        # print(output)
        messages.append({'role': 'user', 'content': output})
    print()
    return list_of_messages

In [36]:
def create_starter_messages(question, index):
    cycle_size = 4
    if False:
        pass
    elif index % cycle_size == 3:
        return [
            {"role": "user", "content": "Translate this problem into Python code.\n\n" + question + "\n\nStart by importing numpy and sympy. If you get a confident answer after running the sympy code, put your final answer within \boxed{}"}
        ]
    elif index % cycle_size == 2:
        return [
            {"role": "user", "content": "Translate the following problem into sympy.\n\n" + question + "\n\nStart by importing sympy. If you get a confident answer after running the sympy code, put your final answer within \boxed{}"}
        ]
    elif index % cycle_size == 1:
        # https://github.com/QwenLM/Qwen2.5-Math?tab=readme-ov-file#-hugging-face-transformers
        return [
            {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
            {"role": "user", "content": question}
        ]
    else:
        # https://github.com/QwenLM/Qwen2.5-Math?tab=readme-ov-file#-hugging-face-transformers
        return [
            {"role": "system", "content": "Please integrate natural language reasoning with Python programs to solve the problem above, and put your final answer within \\boxed{}."},
            {"role": "user", "content": question}
        ]

In [50]:
import re
import polars as pl
import pandas as pd

tool_instruction = '\nPlease solve the problem above, and put your final answer within \\boxed{}. Use math competition strategies to solve these problems and use a proof based problem solving method. From that method, create a python program to solve the problem and execute that to find the solution.'

def predict(id_input, question_input) -> pl.DataFrame:
    """Make a prediction."""
    # Handle both DataFrame and direct string inputs
    if isinstance(id_input, (pl.DataFrame, pd.DataFrame)):
        id_ = id_input[0, 0]
    else:
        id_ = id_input

    if isinstance(question_input, (pl.DataFrame, pd.DataFrame)):
        question = question_input[0, 0]
    else:
        question = question_input

    prompt = question + tool_instruction
    generate_text = generate_text_vllm([prompt], tokenizer, llm)[0]
    print(f"Generated text for ID {id_}: {generate_text}")
    answer = 2
    try:
        result_output = re.findall(r'\\boxed\{(\d+)\}', generate_text)
        if len(result_output) > 0:
            no = naive_parse(result_output[0])
            if len(no) > 0:
                answer = int(no) % 1000
            print(answer)
    except:
        print('error')
    return pl.DataFrame({'id': [id_], 'answer': [answer]})

In [ ]:
import polars as pl

# Load data from reference.csv
data = pl.read_csv('reference.csv')

# Iterate through each row and solve the question using the LLM
results = []
for row in data.iter_rows(named=True):
    id_ = row['id']
    problem = row['problem']
    result = predict(id_, problem)
    results.append(result)

# Combine all results into a single DataFrame
final_results = pl.concat(results)

# Save results to a new CSV file
final_results.write_csv('solutions.csv')

print("Solutions saved to solutions.csv")

Generated text for ID 057f8a:  To determine the greatest positive integer \( d \) for which there will be \( d \) consecutive days without a flight from Dodola island, we need to find the longest gap between consecutive flights of the three companies. The key is to find the least common multiple (LCM) of the intervals at which the planes depart, because the LCM will give us the smallest interval of time after which all three companies will have a flight on the same day.

The intervals are 100 days, 120 days, and 150 days. First, we find the prime factorizations of these numbers:
\[
100 = 2^2 \times 5^2
\]
\[
120 = 2^3 \times 3 \times 5
\]
\[
150 = 2 \times 3 \times 5^2
\]

The LCM is found by taking the highest power of each prime that appears in the factorizations:
\[
\text{LCM} = 2^3 \times 3^1 \times 5^2 = 8 \times 3 \times 25 = 600
\]

This means that every 600 days, all three companies will have a flight on the same day. Therefore, in any 600-day period, there will be at least one